# Terraform prototype

In [136]:
import re
import os
import json
import subprocess

In [137]:
def str_diff(str_a, str_b):

    result_a = ""
    result_b = ""
    max_len=len(str_a) if len(str_a)>len(str_b) else len(str_b)

    for i in range(max_len):
        letter_a = str_a[i:i+1]
        letter_b = str_b[i:i+1]
        if letter_a != letter_b:
            result_a += letter_a
            result_b += letter_b
    
    return result_a, result_b

In [138]:
def get_path(path_line):
    path = re.split("\]\[|\[|\]", path_line)
    del(path[0]); del(path[-1])
    return path

In [139]:
def path_diff(left, right):
    return diff

In [140]:
class SingletonMeta(type):

    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            instance            = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        return cls._instances[cls]

In [141]:
class TfSchema(metaclass=SingletonMeta):

    def __init__(self, path):
        """
        Takes path of the folder where the terraform files placed
        creates tf_schema.json
        """
        self.path      = path
        self.file_name = "tf_schema.json"
        self.schema    = self.__schema()

    def __load(self):
        command = f"terraform -chdir={self.path} providers schema -json > {self.path}/{self.file_name}"
        p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
        p.wait()

    def __schema(self):
        self.__load()
        with open(f"{self.path}/{self.file_name}", "r") as f:
            self._schema = json.load(f)
        return self._schema

In [251]:
class TfSchemaHandler():

    def __init__(self, tf_schema_json, provider="kubernetes"):
        self.provider = provider
        self.schema   = tf_schema_json

    def list_resource_schemas(self):
        resource_objects = self.schema["provider_schemas"][f"registry.terraform.io/hashicorp/{self.provider}"]["resource_schemas"].keys()
        return list(resource_objects)

    def list_required_attributes(self, resource):
        resource = self.schema["provider_schemas"][f"registry.terraform.io/hashicorp/{self.provider}"]["resource_schemas"][resource]

        required_attributes      = []
        lookup_key, lookup_value = "required", "true"
        
        def reverse_lookup(json_tree, json_path):

            for key, value in json_tree.items():
                if isinstance(value, dict):
                    reverse_lookup(json_tree[key], json_path + f"[{key}]")
                    # parent = json_tree[key]
                else:
                    if key == lookup_key:
                        required_attributes.append(json_path)
                    else:
                        continue

        reverse_lookup(resource, "")

        return required_attributes
    
    def show_block(self, resource):
        block = self.schema["provider_schemas"][f"registry.terraform.io/hashicorp/{self.provider}"] \
            ["resource_schemas"][resource]["block"]
        return list(block.keys())
    
    def show_block_types(self, resource):
        block_types = self.schema["provider_schemas"][f"registry.terraform.io/hashicorp/{self.provider}"] \
            ["resource_schemas"][resource]["block"]["block_types"]
        return list(block_types.keys())
    
    def show_metadata_attrs(self, resource):
        attrs = self.schema["provider_schemas"][f"registry.terraform.io/hashicorp/{self.provider}"] \
            ["resource_schemas"][resource]["block"]["block_types"]["metadata"]["block"]["attributes"]
        return list(attrs.keys())
    
    def show_spec_block_types(self, resource):
        spec_block_types = self.schema["provider_schemas"][f"registry.terraform.io/hashicorp/{self.provider}"] \
            ["resource_schemas"][resource]["block"]["block_types"]["spec"]["block"]["block_types"]
        return list(spec_block_types.keys())
    
    def get_parent(self, resource):
        """ 
        Takes schemas object, returns json object
        """
        parent = self.schema["provider_schemas"][f"registry.terraform.io/hashicorp/{self.provider}"] \
            ["resource_schemas"][resource]
        return parent
    
    def get_block(self, resource):
        """ 
        Takes json object, returns json objects
        """
        block = resource.get("block")
        return block
   
    def expand_block(self, resource_object):
        """ 
        Takes json object, returns json attributes and 
        block_type objects if exist
        """
        attributes = resource_object.get("attributes")
        block_types = resource_object.get("block_types")
        return attributes, block_types

## 1. Create terrafrom file with schema

In [252]:
js = TfSchema("./")

## 2. Initialize Handler

In [253]:
k8s = TfSchemaHandler(js.schema)

## 3. Print resources

In [254]:
k8s.list_resource_schemas()

['kubernetes_api_service',
 'kubernetes_certificate_signing_request',
 'kubernetes_cluster_role',
 'kubernetes_cluster_role_binding',
 'kubernetes_config_map',
 'kubernetes_cron_job',
 'kubernetes_csi_driver',
 'kubernetes_daemonset',
 'kubernetes_default_service_account',
 'kubernetes_deployment',
 'kubernetes_endpoints',
 'kubernetes_horizontal_pod_autoscaler',
 'kubernetes_ingress',
 'kubernetes_job',
 'kubernetes_limit_range',
 'kubernetes_manifest',
 'kubernetes_mutating_webhook_configuration',
 'kubernetes_namespace',
 'kubernetes_network_policy',
 'kubernetes_persistent_volume',
 'kubernetes_persistent_volume_claim',
 'kubernetes_pod',
 'kubernetes_pod_disruption_budget',
 'kubernetes_pod_security_policy',
 'kubernetes_priority_class',
 'kubernetes_replication_controller',
 'kubernetes_resource_quota',
 'kubernetes_role',
 'kubernetes_role_binding',
 'kubernetes_secret',
 'kubernetes_service',
 'kubernetes_service_account',
 'kubernetes_stateful_set',
 'kubernetes_storage_class'

## 4. [Optional] load json into object

In [271]:
resource = "kubernetes_pod"

In [272]:
k8s.show_block(resource)

['attributes', 'block_types', 'description_kind']

In [273]:
obj_01 = k8s.get_parent(resource)

In [274]:
data = json.dumps(obj_01)

In [275]:
obj_01 = json.loads(data, object_hook=lambda d: SimpleNamespace(**d))

In [284]:
obj_01.block.block_types.metadata.block.attributes.name.type

'string'

In [285]:
obj_01.block.block_types.metadata.block.attributes.name.optional

True